<a href="https://colab.research.google.com/github/CatMcQueen/catmcqueen.github.io/blob/main/Tutorial_onehot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')
#%cd drive/My Drive/

In [3]:
!git clone -n https://github.com/CatMcQueen/catmcqueen.github.io.git --depth 1


Cloning into 'catmcqueen.github.io'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 12 (delta 2), reused 9 (delta 2), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [4]:
#Then check out just the file you want like so:
%cd catmcqueen.github.io
!git checkout HEAD sherlock.txt

/content/catmcqueen.github.io


In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
stopwords = set(stopwords.words('english')) 
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
file = open('sherlock.txt', mode='r')
# read all lines in sherlock.txt into a list
sherlock_txt = file.read()
 
# make sure to always close the file
file.close()

In [7]:
## First break the text into sentences, removing numbers and unnecessary whitespaces.
#sherlock_txt   = re.sub(r'[0-9]+', ' ', sherlock_txt) # do we want numbers? unclear
sherlock_text1  = re.sub('^\s+', '', sherlock_txt)# get rid of leading whitespace
sherlock_text2  = re.sub('\s+', ' ', sherlock_text1)# replace all tabs/spaces with spaces
sherlock_nopunc = re.sub('[^a-zA-Z ]', '', sherlock_text2) # take out all punctuation


In [53]:
# now that we are broken into sentences using the nltk sentence tolkenizer,
# break each sentence into words

sentences = sent_tokenize(sherlock_text1)
#words    = [word_tokenize(t) for t in sentences]


In [54]:

allwords = word_tokenize(sherlock_nopunc.lower())

In [55]:
# make a list of sentences without the stopwords

clean_sentences = []
for sent in sentences:
  splitsent = sent.split(' ')
  clean_sentences.append([k for k in splitsent if k not in stopwords])


In [58]:
#make a list of words without the stopwords
clean_words = [k for k in allwords if k not in stopwords]

In [62]:

from collections import Counter

def most_frequent(inlist, maxlist=10):
    occurence_count    = Counter(inlist)
    pairs              = occurence_count.most_common(maxlist)#[0][0]

    most_common        = []
    most_common_counts = []
    for x in range(maxlist):
      most_common.append(pairs[x][0])
      most_common_counts.append(pairs[x][1])
    return most_common, most_common_counts

top_words, word_counts = most_frequent(clean_words)
print(top_words)
print(word_counts)

top_sent, sent_count  = most_frequent(sentences)
print(top_sent)
print(sent_count)


['said', 'holmes', 'upon', 'one', 'would', 'man', 'could', 'mr', 'us', 'well']
[2734, 2386, 2302, 2217, 1900, 1826, 1639, 1389, 1340, 1268]
['I asked.', 'he asked.', 'he cried.', 'said he.', 'Holmes?"', '"No."', '"Exactly.', 'I cried.', '"Yes."', 'said Holmes.']
[71, 59, 48, 45, 38, 34, 33, 30, 30, 29]


## Text Generation
We are doing world level LSTM, not character level.

In [15]:
# import what we need to make the RNN/LSTM
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import optimizers
from keras.utils import Sequence
from keras.models import Sequential
from keras.layers  import  Embedding
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.utils import np_utils
import keras.utils as ku 
from sklearn.feature_extraction import DictVectorizer
from typing import Dict, List, Text, Union
import math 

In [16]:
# Retokenize words for word generation
# get all of our words as tokens in a list
allwords      = word_tokenize(sherlock_text2.lower())
vocab         = set(allwords)
VOCAB_SIZE    = len(vocab)

print(VOCAB_SIZE)


20016


In [18]:
# first lets make our sequences
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

# set it up so we can encode our data
encoder      = LabelEncoder()
values       = np.array(allwords)
int_encoded  = encoder.fit_transform(values)

# force it to be sparse until we need it to not in the Generator
ohot_encoder = OneHotEncoder()

#len(int_encoded) is the same length as our len(allwords)
int_encoded  = int_encoded.reshape(len(int_encoded), 1) 
ohot_encoded = ohot_encoder.fit_transform(int_encoded)
#verify it works
idx          = int_encoded[0]
inverted     = encoder.inverse_transform([np.argmax(ohot_encoded[idx, :])])
# and to verify it works, we'll check that the inverted val is
# the same as the index at that location
assert allwords[idx[0]] == inverted[0]

In [20]:
# we have already read in our sherlock.txt and have the data loaded
# now we can generate our own text
SEQ_LENGTH = 4
STEP = 5

In [21]:
# now we're going to make sequences of text, then following text and 
# one hot encode it
# the one hot is too expensive to do in words

def make_sequences(text, seq, step):
        sequences  = []
        next_words = []

        # skip every so step, and collect tokens 
        for i in range(0, VOCAB_SIZE - seq, step):
            cur_words = text[i: i + seq -1]
            sequence = []
            for idx, x in enumerate(cur_words):
                one_hot = ohot_encoded[i + idx]
                sequence.append(one_hot)
            sequences.append(sequence)

            next_word = text[i + seq]
            one_hot = ohot_encoded[i + seq]
            next_words.append(one_hot)
        return sequences, next_words

# too big of a dataset to do all of them, so we're training on a subset, 
# splitting using sequence length( the number of words in the sequence)
# and step (the number of units skipped between each step)
ohot_x, ohot_y = make_sequences(allwords, SEQ_LENGTH, STEP)

In [26]:
#set our model training values
BATCH_SIZE = 1
HIDDEN_DIM = 50 # may need to tune this. Check later
EPOCHS = 30

GENERATE_LENGTH = 20 # we shouldn't need more than 20 words in a sentence, that's a lot


In [23]:
# now let's create our model
optimizer = optimizers.Nadam(lr=.001)
def create_model():
    input_len = SEQ_LENGTH
    model = Sequential()
    #since we know the input size, we can hard code this input shape
    model.add(LSTM(HIDDEN_DIM, input_shape=(SEQ_LENGTH, VOCAB_SIZE)))#, return_sequences=True))

    model.add(Dense(VOCAB_SIZE))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['mean_absolute_error', "accuracy"])
    return model

model = create_model()
# use model.summary() to see the layers and their respective sizes
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                4013400   
_________________________________________________________________
dense (Dense)                (None, 20016)             1020816   
_________________________________________________________________
activation (Activation)      (None, 20016)             0         
Total params: 5,034,216
Trainable params: 5,034,216
Non-trainable params: 0
_________________________________________________________________


In [24]:
## https://github.com/keras-team/keras/issues/9707 says this is how to force
## keras to shuffle data every time manually
## Keras has a native shuffler, but since our output isn't one hot, we need
## to shuffle the data to ensure it does get shuffled. 
class Generator(Sequence):
    # Class is a dataset wrapper for better training performance
    def __init__(self, x_set, y_set, vocab_size, batch_size=128):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.indices = np.arange(self.x.shape[0]) # all possible sequences
        self.vocab_size = vocab_size

    def __len__(self):
        return math.ceil(self.x.shape[0] / self.batch_size)

    def __getitem__(self, idx):
        inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        indexed_x = self.x[inds] # sequences
        indexed_y = self.y[inds] # next_words

        # we need to do batching, so use bool to save space (we don't need an int)
        # it's one hot encoding so bool works fine
        # x =  [batch, seq_length, vocab_size]
        # y = [batch, vocab_size] 
        # for y it's just an array of sparse matricies, so just convert it
        batch_y = np.asarray(indexed_y[0].todense())

        # for x, it's a little harder, since it's an array of 3 dense matricies
        # first convert it to a list, then convert it to a dense matrix, then 
        # convert it back into numpy array
        batch_x = []
        for batchidx, bat in enumerate(indexed_x): # for every seq in the batch
            batch_x.append(bat[0].todense())
        batch_x = np.array(batch_x) 

        return batch_x, batch_y
        
        return batch_x, batch_y
    
    def on_epoch_end(self):
        np.random.shuffle(self.indices)

train_generator = Generator(np.array(ohot_x), np.array(ohot_y), VOCAB_SIZE, batch_size=BATCH_SIZE)

In [27]:
#Keras fit has an inline batch sizer with defaulted shuffle = true, so this should shuffle
# the data without taking too much in at once
model.fit(train_generator, epochs=EPOCHS, verbose=1, batch_size=BATCH_SIZE)

Epoch 1/30
4003/4003 [==============================] - 59s 15ms/step - loss: 6.2988 - mean_absolute_error: 9.8746e-05 - accuracy: 0.0600
Epoch 2/30
4003/4003 [==============================] - 59s 15ms/step - loss: 5.8042 - mean_absolute_error: 9.8534e-05 - accuracy: 0.0597
Epoch 3/30
4003/4003 [==============================] - 59s 15ms/step - loss: 5.6233 - mean_absolute_error: 9.8342e-05 - accuracy: 0.0615
Epoch 4/30
4003/4003 [==============================] - 59s 15ms/step - loss: 5.5163 - mean_absolute_error: 9.8137e-05 - accuracy: 0.0615
Epoch 5/30
4003/4003 [==============================] - 59s 15ms/step - loss: 5.4254 - mean_absolute_error: 9.8026e-05 - accuracy: 0.0622
Epoch 6/30
4003/4003 [==============================] - 59s 15ms/step - loss: 5.3026 - mean_absolute_error: 9.7775e-05 - accuracy: 0.0697
Epoch 7/30
4003/4003 [==============================] - 59s 15ms/step - loss: 5.1452 - mean_absolute_error: 9.7354e-05 - accuracy: 0.0764
Epoch 8/30
4003/4003 [============

In [51]:
# now we use that to generate text
# since we need to input items that are one-hot encoded, we're going
# to use the same method as before to turn it into a one-hot encoding 
# method for generating text
def generate_text(model, length, text_size):
    # first get a random word as our starting point (input)
    ix     = [np.random.randint(text_size-SEQ_LENGTH) + SEQ_LENGTH]
    val    = [np.argmax(ohot_encoded[idx, :])]
    y_char = np.ndarray.tolist(encoder.inverse_transform(val))

    # this is the output text, it'll be length long
    for i in range(length):
        #try:
        # one hot encode the current word
        X_words = []
        for n in range(SEQ_LENGTH):
          indexer = ix[0] - SEQ_LENGTH + n
          X = ohot_encoded[indexer, :]
          X = X.todense()
          X_words.append(X)
        X_words = np.array(X_words)
        #then the index changes to the predicted word and we continue
        ix = [np.argmax(model.predict(X_words, 1))]
        val = [np.argmax(ohot_encoded[ix, :])]
        y_char.append(np.ndarray.tolist(encoder.inverse_transform(val))[0])
        #except:
        #    continue

    # then combine all the words into a sentence        
    return (' ').join(y_char)

generate_text(model, GENERATE_LENGTH, len(allwords))

'must he mixed gregson pass to here said and her i of in and still redskins horsemen falling to for refer'